Imports


In [1]:
import numpy as np
import cv2
import glob
import os
import os, random
import shutil
from tqdm import tqdm
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from os import listdir
from os.path import join, isfile
%matplotlib inline

Function that picks random images from a source directory and copies them to destination directory 

In [3]:
def randomImagePicker(sourceList, destinationList, numberOfImages):

    for src, dst in tqdm(zip(range(len(sourceList)), range(len(destinationList)))):
        src_dir = sourceList[src]
        dst_dir = destinationList[dst]
        #print('src: ', src_dir)
        #print('dst: ', dst_dir)
        file_list = os.listdir(src_dir)
        #print('file list: ', file_list)
        #print(len(file_list))
        random_list = random.sample(file_list, numberOfImages)
        #print('random list: ', random_list)

        for i in range(len(random_list)):
            current_file = random_list[i]
            #print('src: ',src_dir + "/" + current_file)
            #print('dst ',dst_dir  + "/" + current_file)
            shutil.copy(src_dir + "/" + current_file, dst_dir + "/" + current_file)


Convert images to grayscale

In [5]:
def to_gray(pathlist):
    for path in pathlist:
        files = [f for f in listdir(path) if isfile(join(path,f))] 
        #print(path, files)

        for image in files:
            img = os.path.join(path, image)
            img = img.replace('\\','/')
            #print(img)
            try:
                current_img_gray = cv2.imread(img)
                gray = cv2.cvtColor(current_img_gray, cv2.COLOR_BGR2GRAY)
                cv2.imwrite(img, gray)
            except:
                print ("{} is not converted".format(img))

Generate description file

In [22]:
# reads all the files in the /negative folder and generates neg.txt from them.
def generate_negative_description_file(filepath):
    # open the output file for writing. will overwrite all existing data in there
    file = filepath + '/neg.txt'
    print(file)
    with open(file, 'w') as f:
        # loop over all the filenames
        for filename in os.listdir(filepath):
            f.write(filepath + '/' + filename + '\n')

Results function that takes images from the positive directory, draws rectangels around the faces and saves the images with the rectangels

In [6]:
def results(positive_directory, cascade, save):
    for filepath in glob.iglob(positive_directory):
        filepath = filepath.replace('\\','/')
        #print(filepath)
        filename = filepath.split('/')[-1]
        #print(filename)
        img = cv2.imread(filepath)
        model = cv2.CascadeClassifier(cascade)
        faces=model.detectMultiScale(img, scaleFactor=1.4, minNeighbors=3, minSize= [30,30])
        for x, y, w, h in faces:
            img=cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
            resized=cv2.resize(img,(int(img.shape[1]), int(img.shape[0]))) 
            cv2.imshow("Deteced-face",resized)
            savepath = save + filename
            cv2.imwrite(savepath, img)
            cv2.waitKey(1)
            cv2.destroyAllWindows()


Plot results

In [11]:
def plot_results(result_path, positive_class):
    images = []
    for img_path in glob.glob(result_path):
        images.append(mpimg.imread(img_path))

    title_name = result_path.split('/')[-2]
    columns = 5
    count = 1
    
    plt.figure(figsize=(24,24))

    for i, image in enumerate(images[:35]):
        plt.subplot( int(len(images[:35]) / columns + 1), columns, i+1)
        plt.imshow(image)

        plt.axis('off')
        plt.title('Image: ' + str(count))
        plt.tight_layout()
        plt.savefig('save/images/here' + title_name + '.png')  
        count += 1

Compute IoU score

In [4]:
from collections import namedtuple
import numpy as np
import cv2
# define the `Detection` object
# image_path : The path to our input image that resides on disk
# gt : The ground-truth bounding box
# pred : The predicted bounding box from our model
Detection = namedtuple("Detection", ["image_path", "gt", "pred"])
 
def bb_intersection_over_union(boxA, boxB):
	# determine the (x, y)-coordinates of the intersection rectangle
	xA = max(boxA[0], boxB[0])
	yA = max(boxA[1], boxB[1])
	xB = min(boxA[2], boxB[2])
	yB = min(boxA[3], boxB[3])

	# compute the area of intersection rectangle
	interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
	if interArea == 0:
		return 0
	# compute the area of both the prediction and ground-truth
	# rectangles
	boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
	boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

	# compute the intersection over union by taking the intersection
	# area and dividing it by the sum of prediction + ground-truth
	# areas - the interesection area
	iou = interArea / float(boxAArea + boxBArea - interArea)

	# return the intersection over union value
	return iou